# Selektivität: Der kritische Filter
Selektivität bedeutet, dass nur ein winziger Bruchteil der permanent auf uns einströmenden Sinneseindrücke tatsächlich ins Bewusstsein gelangt und überhaupt eine Chance hat, gespeichert zu werden. Unser Gehirn verarbeitet etwa 11 Millionen Bits pro Sekunde an sensorischen Informationen, aber nur etwa 50 Bits schaffen es ins Bewusstsein. Diese massive Filterung verhindert, dass wir im Datenmeer untergehen.

Biologische Systeme haben dafür ausgeklügelte Mechanismen entwickelt: Der Thalamus fungiert als Torwächter, der entscheidet, welche sensorischen Informationen das Bewusstsein erreichen. Die Amygdala verstärkt emotional bedeutsame Ereignisse durch den „Negativity Bias“ – einen evolutionären Vorteil, der bedrohliche Stimuli bevorzugt verarbeitet. Der Hippocampus bindet Ereignisse episodisch (Ort, Zeit, Kontext), während der präfrontale Kortex durch Top-Down-Steuerung Ziele setzt und Aufmerksamkeit lenkt.

Für Multi-Agent-Systeme übersetzt sich dieses Prinzip in intelligente Filterarchitekturen. Diese Systeme müssen in Echtzeit entscheiden, welche Informationen gespeichert, welche verarbeitet und welche verworfen werden – eine Herausforderung, die mit wachsender Datenmenge und Komplexität der Informationsströme überproportional zunimmt.

---

## Selektive Speicherfilterung – Neurobiologische Grundlagen und praktische Anwendung

### Problemstellung und Motivation

In einer Welt rasant wachsender Datenmengen stehen Organisationen vor der zentralen Aufgabe, relevante von irrelevanten Informationen zu unterscheiden. Versicherungsunternehmen verarbeiten täglich Zehntausende Schadensmeldungen, Kundenanfragen und IoT-Sensordaten. Ohne intelligente Filterung droht kognitive Überlastung – sowohl für menschliche Sachbearbeiter als auch für automatisierte Systeme.

Das menschliche Gehirn hat für dieses Problem über Millionen Jahre evolutionär hochoptimierte Mechanismen entwickelt. Von den rund 11 Millionen Bits pro Sekunde sensorischen Inputs erreichen nur etwa 50 Bits pro Sekunde die bewusste Verarbeitung – eine Filterrate von über 99,999 %. Diese massive Reduktion erfolgt durch spezialisierte neuronale Strukturen: Der **Thalamus** fungiert als Torwächter, die **Amygdala** priorisiert emotional relevante Ereignisse (Negativity Bias), der **Hippocampus** bindet Ereignisse episodisch an Kontext, und der **präfrontale Kortex** steuert Aufmerksamkeit durch Top-Down-Signale.

Überträgt man diese Prinzipien auf technische Systeme, entsteht ein mehrstufiges Triage-Modell, das Salienz (Auffälligkeit), Priorität (Relevanzbewertung), zeitliche Dringlichkeit und Kontext kombiniert. Im Versicherungswesen ermöglicht dies, kritische Fälle sofort zu bearbeiten, während Routinefälle automatisiert oder nachrangig behandelt werden. Zugleich lassen sich regulatorische Anforderungen an Nachvollziehbarkeit und Fairness erfüllen.

**Neurobiologischer Referenzpunkt:**  
Mechanismen der selektiven Aufmerksamkeit (Thalamus, Amygdala, Hippocampus, PFC) als biologische Grundlage adaptiver Informationsfilterung.

### Praktische Relevanz im Versicherungswesen

Ein modernes Kfz-Versicherungsunternehmen verarbeitet täglich über 10.000 Schadensmeldungen, rund 50.000 Kundenanfragen und mehr als 100.000 Sensordaten vernetzter Fahrzeuge. Die Herausforderung besteht darin, aus dieser Informationsflut die wirklich relevanten Fälle zu identifizieren: Welche Schadensmeldungen erfordern sofortige Bearbeitung? Welche Muster weisen auf potenziellen Betrug hin? Welche Kundeninteraktionen sollten langfristig gespeichert werden?

Die biologischen Prinzipien lassen sich auf analytische Plattformen übertragen und in vier zentrale Bausteine verdichten:

- **Salienz als Merkmalsbewertung:** Auffällige Ereignisse (hohe Schadenshöhe, Personenschaden, Betrugsrisiko) werden bevorzugt verarbeitet – analog zur thalamischen Filterung irrelevanter Signale.  
- **Priorität als Arousalverstärkung:** Dringliche Fälle (Fristnähe, Kundenwert) erhalten logarithmische Verstärkung – inspiriert durch die Amygdala-Modulation emotionaler Relevanz.  
- **Kontext als episodische Bindung:** Loyalität, Schadenshistorie und VIP-Status fließen in die Bewertung ein – entsprechend der hippocampalen Kontextbindung.  
- **Zeitfaktor als Dringlichkeitssignal:** Ältere unbearbeitete Meldungen werden dringlicher (Dringlichkeitsfunktion: operative Umkehrung der biologischen Vergessenskurve).

Diese methodischen Leitplanken bilden die Grundlage für die anschließende Fallstudie im Versicherungswesen.

---

## Setup und Imports

In [1]:
from __future__ import annotations

import importlib.util
import sys
import warnings
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

warnings.filterwarnings("ignore")

# Notebook-Styling laden
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = (
    NOTEBOOK_DIR.parent
    if (NOTEBOOK_DIR.parent / "notebook_style.py").exists()
    else NOTEBOOK_DIR
)

spec = importlib.util.spec_from_file_location(
    "notebook_style", PROJECT_ROOT / "notebook_style.py"
)
if spec is None or spec.loader is None:
    raise ImportError("notebook_style.py nicht gefunden")

nb_style = importlib.util.module_from_spec(spec)
sys.modules["notebook_style"] = nb_style
spec.loader.exec_module(nb_style)

PLOT_COLORS = nb_style.setup_plot_style(
    aliases={
        'salience': 'primary',
        'stored': 'secondary',
        'rejected': 'quaternary',
        'threshold': 'accent',
    },
    cycle_keys=("primary", "secondary", "accent"),
)

SEED = int(nb_style.SEED)
np.random.seed(SEED)

---

### Selektive Filterung

#### *Salienz $S$: Wie wichtig ist das Ereignis?*
Salienz beschreibt die Auffälligkeit eines Ereignisses. Ein großer Schaden ist auffälliger als ein kleiner. Ein altes Ereignis ist weniger auffällig als ein frisches. Ein hohes Betrugsrisiko ist auffälliger als ein niedriges.

#### *Schwellenwert $θ$: Wann speichern wir?*
Nur Ereignisse mit Salienz ≥ θ werden gespeichert. Ein niedriger Schwellenwert bedeutet: viele Ereignisse werden gespeichert. Ein hoher Schwellenwert bedeutet: nur die wichtigsten Ereignisse werden gespeichert.

#### *Kapazität $K$: Wie viele Ereignisse passen in den Speicher?*
Der Speicher hat begrenzte Kapazität (z.B. K=10 Ereignisse). Wenn der Speicher voll ist und ein neues wichtiges Ereignis kommt, ersetzen wir das Ereignis mit der geringsten Salienz.

#### *Recency $τ$: Wie schnell verblassen alte Ereignisse?*
Alte Ereignisse sind weniger auffällig als frische. Die Zeitkonstante τ steuert, wie schnell die Auffälligkeit mit dem Alter abnimmt. Ein niedriges τ bedeutet: schneller Verfall. Ein hohes τ bedeutet: langsamer Verfall.

---

### Praktisches Szenario: Versicherungs-Schadensmeldungen

**Problem:** Eine Versicherung verarbeitet täglich 10.000 Schadensmeldungen, hat aber nur 200 Sachbearbeiter. Ohne intelligente Filterung können kritische Fälle nicht rechtzeitig bearbeitet werden.

**Lösung:** Intelligente Filterung basierend auf Salienz.

**Features pro Schadensmeldung:**
- **Betrag (amount):** Schadenshöhe in Euro (z.B. 500 - 50.000 €)
- **Alter (age_days):** Wie lange liegt die Meldung vor? (z.B. 0 - 30 Tage)
- **Risiko-Score (risk_score):** Betrugsrisiko (0.0 - 1.0)

**Ziel:** Filtern Sie die wichtigsten Ereignisse basierend auf ihrer Salienz. Speichern Sie nur die Top-K Ereignisse.

### Salienz-Score

**Salienz-Score:**
$$S(t) = w_{\text{amount}} \cdot \text{amount\_norm} + w_{\text{recency}} \cdot \text{recency} + w_{\text{risk}} \cdot \text{risk\_score}$$

**Komponenten:**
- **amount_norm** = min(amount / p95, 1.0) — Normalisierte Schadenshöhe (robust: 95. Perzentil)
- **recency** = exp(−age_days / τ) — Exponentieller Abfall (je älter, desto niedriger)
- **risk_score** ∈ [0,1] — Betrugsrisiko (einfache Heuristik)
- **Gewichte:** w_amount + w_recency + w_risk = 1 (z.B. 0.35 + 0.35 + 0.30)

**Interpretation:**
- S(t) ∈ [0,1] (normalisiert)
- Hohe Salienz = wichtiges Ereignis (speichern)
- Niedrige Salienz = unwichtiges Ereignis (verwerfen)

**Neurobiologisches Mapping:**
- Salienz <-> Hippocampale/Neuromodulatorische Filter
- Dopamin/Noradrenalin <-> Salienz-gesteuerte Speicherung
- Recency/τ <-> Zeitliche Empfindlichkeit der Gedächtnisspur

### Gating-Logik: Speicherung und Ersetzung

**Entscheidungsregel:**
```
if S(t) ≥ θ:
    if len(memory) < K:
        speichere Ereignis
    else:
        if S(t) > min(memory.salience):
            ersetze Ereignis mit geringster Salienz
else:
    verwerfe Ereignis
```

**Kurz gesagt:**
1. Wenn Salienz ≥ Schwellenwert:
   - Wenn Speicher nicht voll: speichere das Ereignis
   - Wenn Speicher voll: ersetze das Ereignis mit der geringsten Salienz (nur wenn das neue Ereignis wichtiger ist)
2. Wenn Salienz < Schwellenwert: verwerfe das Ereignis

**Neurobiologisches Mapping:**
- Gating <-> Hippocampale/Neuromodulatorische Filter
- Kapazität K <-> Ressourcenbeschränkung (begrenzte Speicherkapazität)
- Schwellenwert θ <-> Top-Down-Kontrolle durch präfrontalen Kortex

---

### Implementierung

In [2]:
def simulate_events(n=100, seed=42):
    """Simuliert Ereignis-Stream."""
    rng = np.random.default_rng(seed)
    events = []
    for i in range(n):
        amount = rng.uniform(500, 50000)
        age_days = rng.uniform(0, 30)
        risk_score = rng.uniform(0, 1)
        events.append({'amount': amount, 'age_days': age_days, 'risk_score': risk_score})
    return events


def compute_salience(amount, age_days, risk_score, w, tau, p95):
    """Berechnet Salienz-Score."""
    # Robuste Normierung
    amount_norm = np.clip(amount / p95, 0.0, 1.0)
    # Exponentieller Abfall
    recency = np.exp(-age_days / tau)
    # Gewichtete Summe
    salience = w['amount'] * amount_norm + w['recency'] * recency + w['risk'] * risk_score
    return float(np.clip(salience, 0.0, 1.0))


def gate_memory(events, theta, k, w, tau, p95):
    """Filtert Ereignisse und speichert."""
    memory = []
    stored_indices = []
    salience_scores = []

    for idx, event in enumerate(events):
        s = compute_salience(event['amount'], event['age_days'], event['risk_score'], w, tau, p95)
        salience_scores.append(s)

        if s >= theta:
            if len(memory) < k:
                memory.append((idx, s))
                stored_indices.append(idx)
            else:
                min_idx = np.argmin([m[1] for m in memory])
                if s > memory[min_idx][1]:
                    memory[min_idx] = (idx, s)
                    stored_indices.append(idx)

    return memory, stored_indices, salience_scores


def plot_results(events, memory, salience_scores, theta):
    """Visualisiert Zeitreihe."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7))

    # Plot links: Zeitreihe der Salienz-Scores
    stored_indices = [m[0] for m in memory]
    colors = [PLOT_COLORS['stored'] if i in stored_indices else PLOT_COLORS['rejected'] for i in range(len(events))]
    ax1.scatter(range(len(salience_scores)), salience_scores, c=colors, alpha=0.7, s=50)
    ax1.axhline(theta, color=PLOT_COLORS['threshold'], linestyle='--', lw=2, label=f'Schwelle θ={theta:.2f}')
    ax1.set_xlabel('Ereignis-Index', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Salienz S(t)', fontsize=11, fontweight='bold')
    ax1.set_title('Zeitreihe: Salienz-Scores', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper right', fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(-0.05, 1.05)

    # Plot rechts: Aktueller Speicher (Top-K) als Balken
    if memory:
        memory_sorted = sorted(memory, key=lambda x: x[1], reverse=True)
        indices = [m[0] for m in memory_sorted]
        saliences = [m[1] for m in memory_sorted]
        ax2.barh(range(len(memory_sorted)), saliences, color=PLOT_COLORS['salience'], alpha=0.8)
        ax2.set_yticks(range(len(memory_sorted)))
        ax2.set_yticklabels([f'Ereignis {i}' for i in indices], fontsize=9)
        ax2.set_xlabel('Salienz S(t)', fontsize=11, fontweight='bold')
        ax2.set_title(f'Aktueller Speicher (Top-{len(memory)})', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='x')
        ax2.set_xlim(0, 1.0)
    else:
        ax2.text(0.5, 0.5, 'Speicher leer', ha='center', va='center', fontsize=14, transform=ax2.transAxes)
        ax2.set_title('Aktueller Speicher (leer)', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()
    plt.close(fig)

### Interaktives Beispiel: Selektive Speicherfilterung

In [3]:
# Generiere Ereignis-Stream einmalig
events = simulate_events(n=100, seed=SEED)
p95 = np.percentile([e['amount'] for e in events], 95)


def update_plot(theta=0.5, k=10, tau=10.0, w_amount=0.35, w_recency=0.35):
    """Aktualisiert die Plots basierend auf Parametern."""
    # Normalisiere Gewichte
    w_risk = 1.0 - w_amount - w_recency
    w_risk = max(0.0, min(1.0, w_risk))  # Clamp auf [0,1]
    w = {'amount': w_amount, 'recency': w_recency, 'risk': w_risk}
    # Gewichte auf Summe 1 normieren
    total = w['amount'] + w['recency'] + w['risk']
    if total > 0:
        w = {k: v / total for k, v in w.items()}

    # Filtere Ereignisse
    memory, stored_indices, salience_scores = gate_memory(events, theta, k, w, tau, p95)

    # Visualisiere
    plot_results(events, memory, salience_scores, theta)

    # Statistik
    print("\nStatistik:")
    print(f"  - Gespeicherte Ereignisse: {len(memory)}/{k}")
    print(f"  - Verworfene Ereignisse: {len(events) - len(memory)}/{len(events)}")
    accepted = sum(s >= theta for s in salience_scores)
    print(f"  - Akzeptanzrate (S≥θ): {accepted/len(events)*100:.1f}%")
    print(f"  - Gewichte: w_amount={w_amount:.2f}, w_recency={w_recency:.2f}, w_risk={w_risk:.2f}")


# Interaktive Slider
interact(
    update_plot,
    theta=widgets.FloatSlider(0.5, min=0.0, max=1.0, step=0.05,
                               description='θ (Schwelle):'),
    K=widgets.IntSlider(10, min=5, max=20, step=1,
                         description='K (Kapazität):'),
    tau=widgets.FloatSlider(10.0, min=1.0, max=30.0, step=1.0,
                             description='τ (Recency):'),
    w_amount=widgets.FloatSlider(0.35, min=0.0, max=1.0, step=0.05,
                                  description='w_amount:'),
    w_recency=widgets.FloatSlider(0.35, min=0.0, max=1.0, step=0.05,
                                   description='w_recency:')
)

interactive(children=(FloatSlider(value=0.5, description='θ (Schwelle):', max=1.0, step=0.05), IntSlider(value…

<function __main__.update_plot(theta=0.5, k=10, tau=10.0, w_amount=0.35, w_recency=0.35)>

---

### Interpretation der Ergebnisse

**Was beobachten wir?**

1. **Hoher Schwellenwert (θ = 0.7-1.0):**
   - Wenige Ereignisse werden gespeichert
   - Nur die wichtigsten Fälle landen im Speicher
   - Durchlassrate: niedrig (z.B. 5-10%)

2. **Niedriger Schwellenwert (θ = 0.0-0.3):**
   - Viele Ereignisse werden gespeichert
   - Speicher füllt sich schnell
   - Durchlassrate: hoch (z.B. 50-80%)

3. **Kleine Kapazität (K = 5):**
   - Speicher füllt sich schnell
   - Alte, weniger wichtige Ereignisse werden ersetzt
   - Nur die Top-5 wichtigsten Ereignisse bleiben

4. **Große Kapazität (K = 20):**
   - Speicher füllt sich langsamer
   - Mehr Ereignisse können gespeichert werden
   - Weniger Ersetzungen

5. **Hohe Recency-Gewichtung (w_recency = 0.7):**
   - Frische Ereignisse werden bevorzugt
   - Alte Ereignisse verlieren schnell an Salienz
   - Zeitliche Nähe ist wichtig

6. **Hohe Betrag-Gewichtung (w_amount = 0.7):**
   - Große Schäden werden bevorzugt
   - Kleine Schäden verlieren schnell an Salienz
   - Schadenshöhe ist wichtig

**Praxisleitfaden:**
- **Konservativ:** Hoher θ, kleine K -> Nur kritische Fälle
- **Ausgewogen:** Mittlerer θ, mittleres K -> Balance zwischen Speicher und Filterung
- **Aggressiv:** Niedriger θ, große K -> Viele Fälle werden gespeichert

---

### Neurobiologische Grundlagen

#### Warum funktioniert dieses Modell biologisch?

**1. Salienz-gesteuerte Speicherung <-> Hippocampale/Neuromodulatorische Filter**
- Das Gehirn speichert bevorzugt auffällige (salient) Ereignisse
- Der Hippocampus bindet Ereignisse episodisch (Ort, Zeit, Kontext)
- Neuromodulatoren (Dopamin, Noradrenalin) verstärken die Speicherung wichtiger Ereignisse
- Dieses Modell: S(t) ≥ θ entscheidet über Speicherung

**2. Dopamin/Noradrenalin <-> Salienz-gesteuerte Speicherung**
- Dopamin wird bei überraschenden oder wichtigen Ereignissen freigesetzt
- Noradrenalin verstärkt die Aufmerksamkeit bei Stress oder Gefahr
- Beide Neurotransmitter fördern die Gedächtniskonsolidierung
- Dieses Modell: Hohe Salienz = starkes Dopamin-Signal = Speicherung

**3. Kapazität K <-> Ressourcenbeschränkung**
- Das Gehirn hat begrenzte Speicherkapazität
- Nicht alle Ereignisse können gespeichert werden
- Wichtige Ereignisse verdrängen weniger wichtige
- Dieses Modell: K Ereignisse im Speicher, Ersetzung bei Bedarf

**4. Recency/τ <-> Zeitliche Empfindlichkeit der Gedächtnisspur**
- Frische Ereignisse sind auffälliger als alte
- Die Gedächtnisspur verblasst exponentiell (Ebbinghaus-Vergessenskurve)
- Die Zeitkonstante τ steuert die Geschwindigkeit des Verfalls
- Dieses Modell: recency = exp(−age_days/τ) beschreibt den Verfall

---

### Zusammenfassung
Das Salienz-Filterungs-Modell vereint bewährte Konzepte der Neurowissenschaft – etwa Hippocampus-Dynamik, neuromodulatorische Steuerung und Mechanismen der Gedächtniskonsolidierung – in einer praxisorientierten, technisch umsetzbaren Architektur.